In [19]:
import pandas as pd 
df = pd.DataFrame(pd.read_csv('data/NYC TLC Trip Record.csv'))
df2= df[['passenger_count', 'PULocationID', 'DOLocationID', 'RatecodeID', 'total_amount','trip_distance']]
df2.describe()

C:\Users\attha\AppData\Local\Temp\ipykernel_24240\1443726605.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.DataFrame(pd.read_csv('data/NYC TLC Trip Record.csv'))


,passenger_count,PULocationID,DOLocationID,RatecodeID,total_amount,trip_distance
count,63887.000000,68211.000000,68211.000000,63887.000000,68211.000000,68211.000000
mean,1.315870,98.549735,138.429901,1.117160,21.789377,8.114852
std,0.979054,61.244314,76.761311,1.372913,15.457115,585.105955
min,0.000000,1.000000,1.000000,1.000000,-71.500000,0.000000
25%,1.000000,74.000000,74.000000,1.000000,12.900000,1.110000
50%,1.000000,75.000000,138.000000,1.000000,17.900000,1.850000
75%,1.000000,129.000000,219.000000,1.000000,26.150000,3.210000
max,9.000000,265.000000,265.000000,99.000000,491.000000,120098.840000


In [54]:
pd.set_option('display.max_colwidth', 1)

listItem = []
for col in df.columns :
    listItem.append( [col, df[col].nunique(), df[col].unique()])

tabel1Desc = pd.DataFrame(columns=['Column Name', 'Number of Unique', 'Unique Sample'],
                     data=listItem)
tabel1Desc
# tabel1Desc.drop([1,2], axis=0) # drop date and time karena terlalu panjang

,Column Name,Number of Unique,Unique Sample
0,VendorID,2,"[2, 1]"
1,lpep_pickup_datetime,66575,"[2023-01-01 00:26:10, 2023-01-01 00:51:03, 2023-01-01 00:35:12, 2023-01-01 00:13:14, 2023-01-01 00:33:04, 2023-01-01 00:53:31, 2023-01-01 00:09:14, 2023-01-01 00:11:58, 2023-01-01 00:41:29, 2023-01-01 00:50:32, 2023-01-01 00:16:12, 2023-01-01 00:08:43, 2023-01-01 00:26:32, 2023-01-01 00:18:35, 2023-01-01 00:39:32, 2023-01-01 00:49:34, 2023-01-01 00:10:45, 2023-01-01 00:35:11, 2023-01-01 00:31:06, 2023-01-01 00:14:37, 2023-01-01 00:26:21, 2023-01-01 00:41:56, 2023-01-01 00:32:02, 2023-01-01 00:55:13, 2023-01-01 00:01:31, 2023-01-01 00:22:31, 2023-01-01 00:57:28, 2023-01-01 00:35:21, 2023-01-01 00:52:52, 2023-01-01 00:32:56, 2023-01-01 00:24:01, 2023-01-01 00:46:48, 2023-01-01 00:40:58, 2023-01-01 00:20:23, 2023-01-01 01:49:24, 2023-01-01 00:41:53, 2023-01-01 00:56:26, 2023-01-01 00:33:10, 2023-01-01 00:58:28, 2023-01-01 00:42:49, 2023-01-01 00:48:21, 2023-01-01 00:08:05, 2023-01-01 00:13:47, 2023-01-01 00:16:10, 2023-01-01 00:04:25, 2023-01-01 00:33:38, 2023-01-01 00:46:08, 2023-01-01 01:00:08, 2023-01-01 00:28:04, 2023-01-01 00:23:04, 2023-01-01 00:34:07, 2023-01-01 00:52:03, 2023-01-01 00:25:31, 2023-01-01 00:53:09, 2023-01-01 00:42:37, 2023-01-01 00:39:52, 2023-01-01 00:15:32, 2023-01-01 00:06:48, 2023-01-01 00:57:15, 2023-01-01 00:14:13, 2023-01-01 00:52:48, 2023-01-01 00:34:13, 2023-01-01 00:44:41, 2023-01-01 00:11:12, 2023-01-01 01:44:43, 2023-01-01 01:04:44, 2023-01-01 01:48:11, 2023-01-01 01:11:39, 2023-01-01 01:54:25, 2023-01-01 01:59:10, 2023-01-01 01:22:35, 2023-01-01 01:37:10, 2023-01-01 01:46:22, 2023-01-01 01:59:48, 2023-01-01 01:09:49, 2023-01-01 01:23:35, 2023-01-01 01:53:20, 2023-01-01 01:32:14, 2023-01-01 01:01:05, 2023-01-01 01:34:41, 2023-01-01 01:52:08, 2023-01-01 01:49:28, 2023-01-01 01:03:59, 2023-01-01 01:13:11, 2023-01-01 01:40:06, 2023-01-01 01:33:50, 2023-01-01 01:45:19, 2023-01-01 01:51:29, 2023-01-01 01:34:07, 2023-01-01 01:10:08, 2023-01-01 01:32:17, 2023-01-01 01:48:28, 2023-01-01 01:30:43, 2023-01-01 01:51:10, 2023-01-01 01:09:36, 2023-01-01 01:19:54, 2023-01-01 01:31:10, 2023-01-01 01:42:30, 2023-01-01 00:57:51, 2023-01-01 01:27:36, ...]"
2,lpep_dropoff_datetime,66519,"[2023-01-01 00:37:11, 2023-01-01 00:57:49, 2023-01-01 00:41:32, 2023-01-01 00:19:03, 2023-01-01 00:39:02, 2023-01-01 01:11:04, 2023-01-01 00:26:39, 2023-01-01 00:24:55, 2023-01-01 00:46:26, 2023-01-01 01:13:42, 2023-01-01 00:41:43, 2023-01-01 00:17:08, 2023-01-01 00:45:31, 2023-01-01 00:30:09, 2023-01-01 01:18:06, 2023-01-01 01:08:23, 2023-01-01 00:19:37, 2023-01-01 01:17:35, 2023-01-01 00:42:23, 2023-01-01 00:21:50, 2023-01-01 00:30:26, 2023-01-01 00:46:06, 2023-01-01 00:41:25, 2023-01-01 00:59:47, 2023-01-01 00:16:02, 2023-01-01 00:39:24, 2023-01-01 01:09:31, 2023-01-01 00:53:49, 2023-01-01 01:00:53, 2023-01-01 00:42:53, 2023-01-01 00:32:05, 2023-01-01 01:08:17, 2023-01-01 01:04:32, 2023-01-01 00:24:32, 2023-01-01 01:57:51, 2023-01-01 00:41:58, 2023-01-01 01:14:41, 2023-01-01 00:43:08, 2023-01-01 01:14:46, 2023-01-01 00:51:53, 2023-01-01 01:23:50, 2023-01-01 00:13:26, 2023-01-01 00:25:57, 2023-01-01 00:54:55, 2023-01-01 00:17:50, 2023-01-01 00:38:51, 2023-01-01 00:59:10, 2023-01-01 01:05:46, 2023-01-01 00:43:45, 2023-01-01 23:18:32, 2023-01-01 00:50:01, 2023-01-01 00:57:56, 2023-01-01 00:38:40, 2023-01-01 01:13:35, 2023-01-01 00:47:15, 2023-01-01 00:48:54, 2023-01-01 00:23:03, 2023-01-01 01:08:47, 2023-01-01 01:00:15, 2023-01-01 00:25:43, 2023-01-01 01:21:19, 2023-01-01 00:43:57, 2023-01-01 00:48:26, 2023-01-01 00:38:12, 2023-01-01 01:57:31, 2023-01-01 01:11:23, 2023-01-01 02:05:51, 2023-01-01 01:41:21, 2023-01-01 02:05:19, 2023-01-01 02:11:36, 2023-01-01 01:25:11, 2023-01-01 01:41:30, 2023-01-01 02:04:14, 2023-01-01 02:05:29, 2023-01-01 01:17:55, 2023-01-01 01:53:43, 2023-01-01 02:27:01, 2023-01-01 01:43:55, 2023-01-01 01:13:02, 2023-01-01 02:00:06, 2023-01-01 02:00:57, 2023-01-01 02:13:18, 2023-01-01 01:07:17, 2023-01-01 01:26:01, 2

#### ***Problem Identification***

Bagaimana cara Taxi Drivers membuat rute maksimal untuk mendapat revenue terbanyak. 

Hypothesis':

| Hypothesis                | Data to Analyze                                                                                              |
|------------------------|----------------------------------------------------------------------------------------------------------|
| Pada weekdays daerah pusat perkotaan akan lebih sibuk | Passenger_count, PULocationID, DOLocationID, RatecodeID, TotalAmount |
| Pada weekdays maka pagi dan sore hari, pesanan akan lebih banyak | PULocation, Date, RatecodeID, totalAmount, PULocationID, DOLocationID |
| Total amount pada Dispatch akan lebih sedikit | |


reference = https://medium.com/@fathurizkym27/analysis-of-the-nyc-green-taxi-data-e37640534730

### ***Data Cleaning Framework***
- Datetime - Buat format datetime buat cari outlier
- LocationID = Menghapus LocationID yang lebih dari 263 (secara data hanya ada 263)
- Distance = Menghapus outliers atau yang jarak lebih dari quantile 0.85 (lihat datanya nanti pilih salah satu)
- RatecodeID = 
- Trip Distance = Hapus yang melebihi 10 (sebenernya ini ada outliers tapi yang longtrip bisa dilihat karna kalau diantara 4-10 masih banyak)
- Payment Type = [1.0, 2.0, 3.0, 4.0, 5.0, nan] -- Nan dibuat 5
-  

Add rows
- Duration (dropoff-pickup)
- Average Speed (distance/time)

# ***NYC TLC Trip Data***

Dataset ini mencakup data historis transasksi jasa *Taxi and Limousine Commission* (TLC) yang tersedia pada NYC. 

Untuk informasi lebih lanjut tentang TLC NYC bisa pada laman 
(nyc.gov/site/tlc/index.page)


Translasi pribadi column data

| Feature                | Description                                                                                              |
|------------------------|----------------------------------------------------------------------------------------------------------|
| VendorID               | Nama provider data <br>1 = Creative Mobile Technologies, LLC.<br>2 = VeriFone Inc.                                         |
| lpep_pickup_datetime   | Data argo dimulai <br>The date and time when the meter was engaged.                                                           |
| lpep_dropoff_datetime  | Data argo selesai <br>The date and time when the meter was disengaged.                                                        |
| Passenger_count        | Jumlah penumpang (driver yang menginput - kemungkinan error ada) <br>The number of passengers in the vehicle. This is a driver-entered value.                             |
| Trip_distance          | Jarak trip berdasarkan taximeter <br>The elapsed trip distance in miles was reported by the taximeter.                                      |
| PULocationID           | Lokasi GPS saat taximeter dinyalakan <br> TLC Taxi Zone in which the taximeter was engaged.                                                      |
| DOLocationID           | Lokasi GPS saat taximeter dimatikan <br> TLC Taxi Zone in which the taximeter was disengaged.                                                    |
| RateCodeID             | Tarif akhir yang mempengaruhi total harga argo ke lokasi khusus <br> The final rate code is in effect at the end of the trip.<br>1 = Standard rate<br>2 = JFK<br>3 = Newark<br>4 = Nassau or Westchester<br>5 = Negotiated fare<br>6 = Group ride |
| Store_and_fwd_flag     | Bendera indikasi (store and fwd artinya tidak langsung dikirim) This flag indicates whether the trip record was held in the vehicle memory before sending to the vendor, aka "store and forward," because the vehicle did not have a connection to the server.<br>Y = store and forward trip<br>N = not a store and forward trip |
| Payment_type           | A numeric code signifying how the passenger paid for the trip.<br>1 = Credit card<br>2 = Cash<br>3 = No charge<br>4 = Dispute<br>5 = Unknown <br> 6 = Voided trip (cancellation) |
| Fare_amount            | Biaya pokok dari agro <br>The time-and-distance fare is calculated by the meter. Extra Miscellaneous extras and surcharges. Currently, this only includes the $0.50 and $1 rush hour and overnight charges. |
| MTA_tax       (X)         | $0.50 akan secara otomatis ditambahkan ke biaya perjalanan jika perjalanan taksi berada di dalam batasan wilayah yang berlaku. <br> $0.50 MTA tax that is automatically triggered based on the metered rate in use.                        |
| Improvement_surcharge(X)  | Semacam uang gedung lah wkwk <br> $0.30 improvement surcharge assessed on hailed trips at the flag drop. The improvement surcharge began being levied in 2015. |
| Tip_amount             | Tips, kalo kredit otomatis <br>This field is automatically populated for credit card tips. Cash tips are not included.                 |
| Tolls_amount           | Ya biaya tol <br>The total amount of all tolls paid in the trip.                                                           |
| Total_amount           | Total semua pembayaran (exclude cash tips) <br>The total amount charged to passengers. Does not include cash tips.                                      |
| Trip_type              | A code indicating whether the trip was a street hail or a dispatch that is automatically assigned based on the metered rate in use but can be altered by the driver.<br>1 = Street-hail (panggil dijalan)<br>2 = Dispatch (pesan melalui telpon atau app)|
